In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Germany'
chdir(csv_file_path)

Y = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)
X = X_all['2002-05-01':' 2016-12-31']

In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']

Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']


csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *
(dataset_train,dataset_test,dataset_valid,scaler_X,scaler_Y)=prepare_data(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)

x_final_unsplit = scaler_X.transform(X)
y_final_unsplit = scaler_Y.transform(Y)
dataset_train = np.concatenate((x_final_unsplit,y_final_unsplit), axis=1) # concatenate to evaluate on entire calibration set 
 
iters=3
n_steps_in=30
n_steps_out=1

x_final, y_final = split_sequences_y1(dataset_train, n_steps_in, n_steps_out)

In [3]:
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [5]:
test_runs = 100 # to obtain prediction intervals based on Monte Carlo dropout
    
#load model 
csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)

transformer_model = keras.models.load_model('TF_Germany_tune.h5')
transformer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        906         layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 9)        0           multi_head_attention[0][0]   

## Prediction Transformer

In [6]:
print(x_final.shape)
Y_pred_test_tr = np.empty((x_final.shape[0],n_steps_out,test_runs))
y_correct_utf = scaler_Y.inverse_transform(y_final)
for j in range(test_runs): 
    Y_pred_test_tr[:,:,j]=transformer_model.predict(x_final)
    yhat_utf = scaler_Y.inverse_transform(Y_pred_test_tr[:,:,j])
    #print(j)

(5330, 30, 9)


## POC - Percentage of coverage (UQ)

In [9]:
Y_pred = np.mean(Y_pred_test_tr, axis=2)
Y_std = np.std(Y_pred_test_tr, axis=2)

zT = 10.5
T_ival = [Y_pred - zT*Y_std,Y_pred + zT*Y_std]

def calc_POC(list_of_bounds, measured):
    temp = 0 
    for i in range(len(measured)):
        if measured[i]< list_of_bounds[1][i] and measured[i]> list_of_bounds[0][i]:
            temp = temp + 1
    POC = temp/len(measured)
    return POC

print(T_ival[1].shape)
POC_tr = calc_POC(T_ival,y_final) # adapt zT until POC of > 0.95 is reached
print(POC_tr)

(5330, 1)
0.9523452157598499


In [10]:
Y_pred_test_tr_flat = Y_pred_test_tr.reshape((test_runs*Y_pred_test_tr.shape[0], 1))
Y_pred_test_utf_flat = scaler_Y.inverse_transform(Y_pred_test_tr_flat)
Y_pred_test_utf = Y_pred_test_utf_flat.reshape((Y_pred_test_tr.shape[0], 1,test_runs))
Y_pred_calib = np.mean(Y_pred_test_utf, axis=2)
Y_std_calib = np.std(Y_pred_test_utf, axis=2)
T_ival_calib = [Y_pred_calib - zT*Y_std_calib,Y_pred_calib + zT*Y_std_calib] # prediction interval
T_ival_calib[1]

(5330, 1, 100)


array([[375.05860474],
       [375.03509716],
       [375.03450754],
       ...,
       [375.03763874],
       [375.11698507],
       [375.11337174]])

In [11]:
# test data 
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Germany'
chdir(csv_file_path)

X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

X_finaltest = X_all['2017-01-01':]
X_finaltest

,rr,tg,tn,tx,pp,hu,fg,qq,et
time,,,,,,,,,
2017-01-01,0.000000,-3.89,-4.35,-3.360000,1025.30000,94.366670,1.14,45.0,0.337962
2017-01-02,1.400000,-1.23,-3.25,-0.190000,1022.60004,94.200005,1.95,17.0,0.143527
2017-01-03,0.000000,-0.37,-1.48,0.380000,1026.30000,88.470000,3.32,28.0,0.244327
2017-01-04,5.000000,0.28,-0.83,0.600000,1015.10004,93.211770,6.33,18.0,0.162267
2017-01-05,0.000000,-4.26,-13.19,0.000000,1026.80000,78.315000,4.23,37.0,0.272975
...,...,...,...,...,...,...,...,...,...
2021-12-27,1.100000,1.30,0.72,2.920000,1009.50000,94.450005,2.49,27.0,0.254015
2021-12-28,18.300001,3.75,1.42,8.679999,1004.00000,93.175000,3.34,17.0,0.175350
2021-12-29,6.600000,7.51,6.62,8.940000,1007.10004,87.680000,4.40,22.0,0.255928


In [12]:
X_finaltest_trf = scaler_X.transform(X_finaltest)
X_ftest = split_sequences_test(X_finaltest_trf,n_steps_in, n_steps_out)
X_ftest.shape

(1797, 30, 9)

In [13]:
print(x_final.shape)
Y_pred_ftest_tr = np.empty((X_ftest.shape[0],n_steps_out,test_runs))
for j in range(test_runs): 
    Y_pred_ftest_tr[:,:,j]=transformer_model.predict(X_ftest)

(5330, 30, 9)


In [16]:
Y_pred_ftest_tr_flat = Y_pred_ftest_tr.reshape((100*Y_pred_ftest_tr.shape[0], 1))
Y_pred_ftest_utf_flat = scaler_Y.inverse_transform(Y_pred_ftest_tr_flat)
Y_pred_ftest_utf = Y_pred_ftest_utf_flat.reshape((Y_pred_ftest_tr.shape[0], 1,100))

yhat_mean = np.mean(Y_pred_ftest_utf, axis=2)
yhat_std = np.std(Y_pred_ftest_utf, axis=2)
T_ival_utf = [yhat_mean - zT*yhat_std,yhat_mean + zT*yhat_std]
yhat_mean.shape

In [18]:
# files to save forecasts
final_idx_calib = pd.date_range(start='2002-05-30', end='2016-12-31')
final_idx_test = pd.date_range(start='2017-01-30', end='2021-12-31')
final_df_calib = pd.DataFrame(index=final_idx_calib)
final_df_test = pd.DataFrame(index=final_idx_test)
final_df_test.shape

(1797, 0)

In [19]:
final_df_calib['head forecast'] = Y_pred_calib
final_df_calib['95% lower'] = T_ival_calib[0]
final_df_calib['95% upper'] = T_ival_calib[1]
final_df_calib

(5330, 1)


,head forecast,95% lower,95% upper
2002-05-30,374.656903,374.255202,375.058605
2002-05-31,374.658778,374.282459,375.035097
2002-06-01,374.646727,374.258946,375.034508
2002-06-02,374.630646,374.245218,375.016074
2002-06-03,374.621784,374.313412,374.930157
...,...,...,...
2016-12-27,374.607970,374.199614,375.016325
2016-12-28,374.603254,374.185804,375.020705
2016-12-29,374.612631,374.187623,375.037639
2016-12-30,374.630363,374.143741,375.116985


In [20]:
final_df_test['head forecast'] = yhat_mean
final_df_test['95% lower'] = T_ival_utf[0]
final_df_test['95% upper'] = T_ival_utf[1]
final_df_test

,head forecast,95% lower,95% upper
2017-01-30,374.835514,374.420250,375.250779
2017-01-31,374.843888,374.337744,375.350031
2017-02-01,374.829399,374.360702,375.298096
2017-02-02,374.847125,374.417832,375.276418
2017-02-03,374.848824,374.360379,375.337269
...,...,...,...
2021-12-27,374.780324,374.225704,375.334945
2021-12-28,374.819677,374.270905,375.368450
2021-12-29,374.827376,374.304624,375.350129
2021-12-30,374.806514,374.262204,375.350825


In [21]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/submissions'
chdir(csv_file_path)
final_df_calib.to_csv('Germany_calib.csv', float_format='%.2f')
final_df_test.to_csv('Germany_test.csv', float_format='%.2f')